# 3d mitochondria visualizer using vtk 
- Includes a Neuroglancer link generator and a vtk viewer centered on a mitoid of interest 
- This notebook is modified from Allen Institute tutorial on working with Meshes by Forrest Collman  
The original tutorial is located here:   https://github.com/AllenInstitute/MicronsBinder/blob/master/notebooks/intro/MeshExample.ipynb

## Python Requirements
You will need to run this notebook in a Python 3.7 environment  
Sometime meshparty will have a conflict with other tools and not work properly in Anaconda  
If that happens, you will need to start over with a new Anaconda environment  
Install allensdk first, then meshparty, then any other other tools desired  
The installation described below worked well:  
* Install new environment v3.7 in Anaconda
* conda install jupyter
* pip install allensdk
* pip install meshparty
* pip install caveclient
* pip install 'itkwidgets[notebook]>=1.0a8'
* pip install --upgrade --pre itk
* pip install gdown

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils

In [3]:
import vtk

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/pni_mito_cellswskel_v185_fullstats.csv')
mito

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
0,1783325,527936,75186,43844,1430,74824,43668,1323,75472,44120,1561,648518346349523993,300744,175376,57200,6.187842,20.482515,Basal
1,3272360,354680,101562,47000,241,101260,46750,191,101750,47228,303,648518346349538711,406248,188000,9640,3.780227,10.346855,Apical
2,1776789,523452,74500,43210,1457,73444,42632,1401,75636,43732,1564,648518346349538102,298000,172840,58280,6.992364,30.063988,Apical
3,2158922,4428,81214,54992,1230,81172,54962,1228,81254,55018,1235,648518346349538730,324856,219968,49200,0.129240,2.652803,Apical
4,2158896,11196,81194,53124,2007,81142,53096,2004,81250,53168,2014,648518346349532295,324776,212496,80280,0.224896,2.186498,Axonal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210970,1302900,2116324,65836,40492,311,65132,40146,191,66430,40734,368,648518346349516055,263344,161968,12440,20.038195,43.285222,Unknown
210971,3189781,2193980,98800,58132,469,98240,57710,379,99286,58468,536,648518346349539215,395200,232528,18760,21.290737,48.309769,Unknown
210972,2657020,2453896,88816,68882,429,88220,68540,21,89432,69438,877,648518346349539376,355264,275528,17160,25.841965,69.054582,Basal
210973,1307970,2617596,66122,41208,293,65406,40902,173,66600,41576,356,648518346349516055,264488,164832,11720,27.147893,70.360888,Unknown


In [6]:
pyr = pd.read_csv('data/pycids_v185.csv', names=['cellid'], header=None)
pyr.head()

,cellid
0,648518346349539333
1,648518346349539334
2,648518346349523975
3,648518346349539336
4,648518346349538312


In [7]:
pyr_list = pyr.cellid.to_list()
print(pyr_list)

[648518346349539333, 648518346349539334, 648518346349523975, 648518346349539336, 648518346349538312, 648518346349538317, 648518346349523981, 648518346349537297, 648518346349539347, 648518346349537300, 648518346349532180, 648518346349523993, 648518346349538332, 648518346349539359, 648518346349535264, 648518346349538336, 648518346349539366, 648518346349539368, 648518346349533227, 648518346349539376, 648518346349537331, 648518346349535286, 648518346349531192, 648518346349536312, 648518346349538365, 648518346349539390, 648518346349538368, 648518346349535300, 648518346349533252, 648518346349539401, 648518346349538378, 648518346349534289, 648518346349533267, 648518346349538387, 648518346349496405, 648518346349539414, 648518346349523030, 648518346349534292, 648518346349538391, 648518346349538395, 648518346349531230, 648518346349539423, 648518346349524063, 648518346349536354, 648518346349537380, 648518346349503588, 648518346349539430, 648518346349537385, 648518346349539433, 648518346349539435,

In [8]:
# find index number for a pyramidal cellid of interest

pyr.index[pyr.cellid == 648518346349539853]

# 648518346349537741 index 165
# 648518346349538440 index 68
# 648518346349539853 index 195

Int64Index([195], dtype='int64')

# Use index value to view a pyr neuron from list 

In [9]:
# pull all mitos from a cellid of interest from astrocyte list, by index number

indx = 195
cellid = pyr_list[indx]

mito_querydf = mito[mito['cellid'] == cellid]
mito_querydf.sort_values(by='mito_vx', ascending=False)


,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
2730,846898,5241524,69066,37174,1711,57952,35660,1503,79204,40520,1814,648518346349539853,276264,148696,68440,63.240397,221.818001,Basal
22179,3599973,3377304,113472,41788,1675,107570,39978,1384,119166,44296,2148,648518346349539853,453888,167152,67000,41.869051,155.048379,Basal
79301,3214422,3205284,100036,65934,105,98990,61590,21,100752,70056,193,648518346349539853,400144,263736,4200,33.378725,87.217650,Basal
107770,3109814,2925992,97696,71850,694,97028,68568,608,98728,75582,810,648518346349539853,390784,287400,27760,27.976834,61.627937,Basal
20157,2371418,2922848,86616,43976,1499,85220,41896,1168,87786,46970,1845,648518346349539853,346464,175904,59960,30.313414,78.563398,Basal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12226,2746120,1464,91326,56156,1615,91314,56116,1614,91342,56188,1618,648518346349539853,365304,224624,64600,0.072949,4.364229,Basal
11611,3064205,1376,96664,50366,681,96650,50352,680,96680,50386,683,648518346349539853,386656,201464,27240,0.044357,1.110671,Somatic
11561,3068729,1368,96690,52016,994,96672,52002,992,96718,52032,997,648518346349539853,386760,208064,39760,0.057651,2.467077,Somatic
6215,3069566,1356,97638,52794,744,97620,52786,743,97658,52808,747,648518346349539853,390552,211176,29760,0.046990,1.359613,Somatic


In [10]:
cellid

648518346349539853

In [11]:
mito.compartment.unique()

array(['Basal', 'Apical', 'Axonal', 'Unknown', 'Somatic',
       'Unknown dendritic'], dtype=object)

In [12]:
mito_querydf.compartment.unique()

array(['Axonal', 'Somatic', 'Basal', 'Apical', 'Unknown'], dtype=object)

In [13]:
mito_query_basal = mito_querydf[mito_querydf.compartment == 'Basal']
mito_query_basal

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
717,3970091,332188,114540,58920,1279,114014,58698,1272,114964,59126,1287,648518346349539853,458160,235680,51160,4.249911,16.760998,Basal
969,3620264,1832860,109466,53436,650,107652,52536,535,111472,54400,714,648518346349539853,437864,213744,26000,17.480647,38.312640,Basal
1003,3515725,1089960,106290,59742,1079,105440,58126,968,106992,61020,1173,648518346349539853,425160,238968,43160,12.635230,40.912546,Basal
1592,2856982,1533064,92422,58566,1901,91500,57178,1708,93150,59752,2070,648518346349539853,369688,234264,76040,15.310739,36.795616,Basal
1994,2273496,632496,83074,55104,790,82164,54990,767,84360,55262,802,648518346349539853,332296,220416,31600,8.118143,32.224247,Basal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209008,3406849,549656,102902,58438,1842,102498,58118,1758,103368,58648,1914,648518346349539853,411608,233752,73680,5.433149,12.790898,Basal
209357,2038397,581004,78606,53726,93,78204,53178,34,79058,54098,162,648518346349539853,314424,214904,3720,5.588149,12.455862,Basal
210423,2264009,768908,82692,50680,689,81984,50518,573,83166,50866,786,648518346349539853,330768,202720,27560,8.105520,21.703113,Basal
210578,3511865,836376,104410,56030,2010,103808,55838,1891,105332,56156,2125,648518346349539853,417640,224120,80400,8.365461,20.164838,Basal


In [14]:
mito_query_apical = mito_querydf[mito_querydf.compartment == 'Apical']
mito_query_apical

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
6750,2940319,607848,95058,45756,522,94680,44860,490,95698,47094,571,648518346349539853,380232,183024,20880,7.368329,26.088272,Apical
12221,3060500,653216,98562,47898,343,97616,47498,231,99284,48278,439,648518346349539853,394248,191592,13720,7.932234,28.183901,Apical
16895,3059280,2308,96184,48092,562,96160,48060,561,96214,48122,564,648518346349539853,384736,192368,22480,0.091056,3.414956,Apical
21091,3167240,3268,99094,47668,290,99078,47640,288,99116,47702,294,648518346349539853,396376,190672,11600,0.108016,2.843376,Apical
23672,2930010,293436,94504,40458,180,94278,40068,141,94796,40834,218,648518346349539853,378016,161832,7200,3.432485,11.316895,Apical
24822,3271215,154032,100020,47078,169,99848,46914,157,100240,47244,187,648518346349539853,400080,188312,6760,1.633818,4.429118,Apical
26724,2936511,328820,95884,42174,429,95570,41854,419,96342,42652,446,648518346349539853,383536,168696,17160,3.920266,13.426366,Apical
33547,3272205,7112,101142,45706,122,101120,45688,118,101164,45728,126,648518346349539853,404568,182824,4880,0.132928,1.118909,Apical
40504,2944850,9608,94706,45080,460,94662,45058,458,94752,45108,464,648518346349539853,378824,180320,18400,0.188264,1.741668,Apical
42448,2594942,10328,87950,42086,111,87890,42060,106,88014,42112,115,648518346349539853,351800,168344,4440,0.232345,2.833337,Apical


In [15]:
mito_query_axonal = mito_querydf[mito_querydf.compartment == 'Axonal']
mito_query_axonal

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
259,3098978,30152,95960,67402,621,95872,67374,614,96072,67434,631,648518346349539853,383840,269608,24840,0.486600,3.053596,Axonal
4000,871847,50056,58596,46644,2059,58468,46614,2055,58726,46676,2066,648518346349539853,234384,186576,82360,0.653946,2.689317,Axonal
5387,2654589,12524,88578,67446,1206,88506,67416,1203,88644,67488,1211,648518346349539853,354312,269784,48240,0.264417,2.839935,Axonal
11025,3205687,1292,97828,65268,1301,97808,65238,1301,97846,65296,1303,648518346349539853,391312,261072,52040,0.053672,2.231715,Axonal
18639,3224852,32908,99718,71830,789,99660,71796,784,99784,71864,795,648518346349539853,398872,287320,31560,0.398134,1.404149,Axonal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130402,3226346,61564,99042,73044,1475,98942,72904,1468,99096,73188,1481,648518346349539853,396168,292176,59000,0.842898,3.807140,Axonal
130828,3079935,62012,96582,56956,1096,96500,56842,1092,96720,57068,1102,648518346349539853,386328,227824,43840,0.814145,3.381292,Axonal
132905,3077477,64316,96394,55784,1064,96326,55640,1043,96474,55974,1076,648518346349539853,385576,223136,42560,1.006188,5.933732,Axonal
140476,1924217,73388,75320,56806,989,75172,56732,986,75482,56858,993,648518346349539853,301280,227224,39560,0.874900,2.996078,Axonal


In [16]:
mito_query_somatic = mito_querydf[mito_querydf.compartment == 'Somatic']
mito_query_somatic

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
525,3170590,125152,97918,50832,954,97666,50600,937,98176,51040,968,648518346349539853,391672,203328,38160,1.878677,10.200201,Somatic
1525,3069409,137572,97560,51938,651,97436,51722,629,97640,52206,675,648518346349539853,390240,207752,26040,1.871063,8.339374,Somatic
1701,3069550,914524,97728,52570,695,97606,52146,651,97872,53026,719,648518346349539853,390912,210280,27800,6.088356,6.501867,Somatic
2175,3063552,707920,96168,50894,685,95966,49938,661,96376,51638,732,648518346349539853,384672,203576,27400,8.486428,29.385601,Somatic
2358,3171318,151796,98484,51224,897,98320,50948,854,98728,51580,916,648518346349539853,393936,204896,35880,2.703941,20.672793,Somatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206217,3068311,414872,96432,52122,681,96214,51726,652,96696,52404,717,648518346349539853,385728,208488,27240,6.364103,36.083742,Somatic
206896,3063250,439224,96222,50698,960,95636,50486,930,96830,50894,982,648518346349539853,384888,202792,38400,5.306979,18.668014,Somatic
208495,3175488,515060,98326,51564,1005,97814,51368,959,98816,51984,1024,648518346349539853,393304,206256,40200,7.121527,32.804381,Somatic
210175,3175427,697804,98004,52030,677,97756,51616,641,98292,52408,729,648518346349539853,392016,208120,27080,8.710557,32.703864,Somatic


In [17]:
mito_query_unknown = mito_querydf[mito_querydf.compartment == 'Unknown']
mito_query_unknown

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
22672,3059190,3700,96068,47866,623,96036,47836,621,96092,47892,626,648518346349539853,384272,191464,24920,0.095049,1.511368,Unknown
53911,3072741,36016,96266,55164,1004,96234,54972,1000,96308,55360,1008,648518346349539853,385064,220656,40160,0.658849,5.312448,Unknown
79897,2842475,336252,93390,50212,1076,93058,50082,1053,93738,50348,1093,648518346349539853,373560,200848,43040,2.980624,5.643130,Unknown
127401,3059068,58488,95982,47734,609,95886,47668,598,96070,47792,619,648518346349539853,383928,190936,24360,0.912092,5.344543,Unknown
190918,3285304,218232,100190,51314,1130,99898,51212,1106,100512,51480,1150,648518346349539853,400760,205256,45200,2.668829,9.617312,Unknown


In [18]:
mito_query_unkdend = mito_querydf[mito_querydf.compartment == 'Unknown dendritic']
mito_query_unkdend

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment


In [19]:
mito_query_basal_mitolist = mito_query_basal.mito_id.to_list()
print(f"number of basal mitochondria: "+str(len(mito_query_basal_mitolist)))
# print(f"type: "+str(type(mito_query_basal_mitolist)))
# print('')
# print(mito_query_basal_mitolist) # uncomment to print the mito id list

mito_query_apical_mitolist = mito_query_apical.mito_id.to_list()
print(f"number of apical mitochondria: "+str(len(mito_query_apical_mitolist)))
#print(f"type: "+str(type(mito_query_apical_mitolist)))
#print('')
# print(mito_query_apical_mitolist) # uncomment to print the mito id list

mito_query_axonal_mitolist = mito_query_axonal.mito_id.to_list()
print(f"number of axonal mitochondria: "+str(len(mito_query_axonal_mitolist)))
#print(f"type: "+str(type(mito_query_axonal_mitolist)))
#print('')
# print(mito_query_axonal_mitolist) # uncomment to print the mito id list

mito_query_somatic_mitolist = mito_query_somatic.mito_id.to_list()
print(f"number of somatic mitochondria: "+str(len(mito_query_somatic_mitolist)))
#print(f"type: "+str(type(mito_query_somatic_mitolist)))
#print('')
# print(mito_query_somatic_mitolist) # uncomment to print the mito id list

mito_query_unknown_mitolist = mito_query_unknown.mito_id.to_list()
print(f"number of unknown mitochondria: "+str(len(mito_query_unknown_mitolist)))
#print(f"type: "+str(type(mito_query_unknown_mitolist)))
#print('')
# print(mito_query_unknown_mitolist) # uncomment to print the mito id list

mito_query_unkdend_mitolist = mito_query_unkdend.mito_id.to_list()
print(f"number of unknown dendritic mitochondria: "+str(len(mito_query_unkdend_mitolist)))
#print(f"type: "+str(type(mito_query_unkdend_mitolist)))
#print('')
# print(mito_query_unkdend_mitolist) # uncomment to print the mito id list

number of basal mitochondria: 317
number of apical mitochondria: 55
number of axonal mitochondria: 89
number of somatic mitochondria: 479
number of unknown mitochondria: 5
number of unknown dendritic mitochondria: 0


# vtk 3d viewer

In [20]:
# setup the mesh meta to handle downloads and caching
mesh_dir = 'data/neuron_meshes_v185/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/'
mito_source = "precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [21]:
cell_id = cellid

In [22]:
# If you get an error, run the next cell to download the mesh, then rerun this cell again
seg_id = cell_id
mesh_file = os.path.join(mesh_dir, str(seg_id)+'.h5')
mesh = mm.mesh(filename = mesh_file)

In [23]:
# If you get an error on the previous cell, run this cell
# After the mesh is successfully downloaded, rerun the previous cell 
downloadmesh = mm.mesh(seg_id = seg_id, remove_duplicate_vertices=True)

- Note: The first time you download the meshes will take a long time, which gets longer the more mitos are in the cell of interest
- After downloading the meshes, this cell will run much faster subsequently

In [24]:
# download the mito meshes by compartment for this cell

for i in range(len(mito_query_basal_mitolist)):
    mito_id = mito_query_basal_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_apical_mitolist)):
    mito_id = mito_query_apical_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)

for i in range(len(mito_query_axonal_mitolist)):
    mito_id = mito_query_axonal_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_somatic_mitolist)):
    mito_id = mito_query_somatic_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_unknown_mitolist)):
    mito_id = mito_query_unknown_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)
    
for i in range(len(mito_query_unkdend_mitolist)):
    mito_id = mito_query_unkdend_mitolist[i]
    mito_seg_id = mito_id
    mito_downloadmesh = mito_mm.mesh(seg_id = mito_seg_id, remove_duplicate_vertices=True)

In [25]:
mito_query_basal_mitolist[0]

3970091

In [26]:
# iterate the mito_mesh for each mito
# from https://python-forum.io/thread-23500.html

var_iterator_basal = {}
for i in range(len(mito_query_basal_mitolist)):
    mito_seg_id = mito_query_basal_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_basal['mito_mesh_' + str(mito_query_basal_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_basal)

var_iterator_apical = {}
for i in range(len(mito_query_apical_mitolist)):
    mito_seg_id = mito_query_apical_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_apical['mito_mesh_' + str(mito_query_apical_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_apical)

var_iterator_axonal = {}
for i in range(len(mito_query_axonal_mitolist)):
    mito_seg_id = mito_query_axonal_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_axonal['mito_mesh_' + str(mito_query_axonal_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_axonal)

var_iterator_somatic = {}
for i in range(len(mito_query_somatic_mitolist)):
    mito_seg_id = mito_query_somatic_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_somatic['mito_mesh_' + str(mito_query_somatic_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_somatic)

var_iterator_unknown = {}
for i in range(len(mito_query_unknown_mitolist)):
    mito_seg_id = mito_query_unknown_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_unknown['mito_mesh_' + str(mito_query_unknown_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_unknown)

var_iterator_unkdend = {}
for i in range(len(mito_query_unkdend_mitolist)):
    mito_seg_id = mito_query_unkdend_mitolist[i] 
    mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
    var_iterator_unkdend['mito_mesh_' + str(mito_query_unkdend_mitolist[i])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(var_iterator_unkdend)


In [27]:
len(var_iterator_apical)

55

In [28]:
var_iterator_apical

{'mito_mesh_2940319': <trimesh.Mesh(vertices.shape=(2242, 3), faces.shape=(4480, 3))>,
 'mito_mesh_3060500': <trimesh.Mesh(vertices.shape=(2213, 3), faces.shape=(4422, 3))>,
 'mito_mesh_3059280': <trimesh.Mesh(vertices.shape=(35, 3), faces.shape=(70, 3))>,
 'mito_mesh_3167240': <trimesh.Mesh(vertices.shape=(72, 3), faces.shape=(144, 3))>,
 'mito_mesh_2930010': <trimesh.Mesh(vertices.shape=(1038, 3), faces.shape=(2068, 3))>,
 'mito_mesh_3271215': <trimesh.Mesh(vertices.shape=(528, 3), faces.shape=(1052, 3))>,
 'mito_mesh_2936511': <trimesh.Mesh(vertices.shape=(1164, 3), faces.shape=(2328, 3))>,
 'mito_mesh_3272205': <trimesh.Mesh(vertices.shape=(35, 3), faces.shape=(66, 3))>,
 'mito_mesh_2944850': <trimesh.Mesh(vertices.shape=(63, 3), faces.shape=(122, 3))>,
 'mito_mesh_2594942': <trimesh.Mesh(vertices.shape=(84, 3), faces.shape=(164, 3))>,
 'mito_mesh_2940795': <trimesh.Mesh(vertices.shape=(674, 3), faces.shape=(1352, 3))>,
 'mito_mesh_2936273': <trimesh.Mesh(vertices.shape=(1052, 3), 

In [29]:
mito_query_apical_mitolist[0]

2940319

In [34]:
# opacity and color settings for cell membrane and mito meshes
cell_opac = 0.075 # 0.075
cell_color = (1, 1, 1) # (0.2, 0.4, 0.7) blue
mito_opac_basal = 0.25
mito_color_basal = (0, 0.75, 1) # sky blue
mito_opac_apical = 0.75
mito_color_apical = (0.9, 0.9, 0) # gold 
mito_opac_axonal = 1
mito_color_axonal = (0, 0, 0) # black
mito_opac_somatic = 0.25
mito_color_somatic = (0.7, 0.4, 0.7) # purple
mito_opac_unknown = 1
mito_color_unknown = (1, 0, 0) # red
mito_opac_unkndend = 1
mito_color_unkdend = (0.9, 0.6, 0) # orange

# cell membrane mesh
mesh_actor = trimesh_vtk.mesh_actor(mesh, opacity=cell_opac, color=cell_color)

# mito meshes
actor_iterator_basal = {}
for i in range(len(mito_query_basal_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_basal['mito_mesh_actor_' + str(mito_query_basal_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_basal['mito_mesh_'+str(mito_query_basal_mitolist[i])], opacity=mito_opac_basal, color=(mito_color_basal))
locals().update(actor_iterator_basal)

actor_iterator_apical = {}
for i in range(len(mito_query_apical_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_apical['mito_mesh_actor_' + str(mito_query_apical_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_apical['mito_mesh_'+str(mito_query_apical_mitolist[i])], opacity=mito_opac_apical, color=(mito_color_apical))
locals().update(actor_iterator_apical)

actor_iterator_axonal = {}
for i in range(len(mito_query_axonal_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_axonal['mito_mesh_actor_' + str(mito_query_axonal_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_axonal['mito_mesh_'+str(mito_query_axonal_mitolist[i])], opacity=mito_opac_axonal, color=(mito_color_axonal))
locals().update(actor_iterator_axonal)

actor_iterator_somatic = {}
for i in range(len(mito_query_somatic_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_somatic['mito_mesh_actor_' + str(mito_query_somatic_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_somatic['mito_mesh_'+str(mito_query_somatic_mitolist[i])], opacity=mito_opac_somatic, color=(mito_color_somatic))
locals().update(actor_iterator_somatic)

actor_iterator_unknown = {}
for i in range(len(mito_query_unknown_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_unknown['mito_mesh_actor_' + str(mito_query_unknown_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_unknown['mito_mesh_'+str(mito_query_unknown_mitolist[i])], opacity=mito_opac_unknown, color=(mito_color_unknown))
locals().update(actor_iterator_unknown)

actor_iterator_unkdend = {}
for i in range(len(mito_query_unkdend_mitolist)):
    #random_color = list(np.random.random(size=3))
    actor_iterator_unkdend['mito_mesh_actor_' + str(mito_query_unkdend_mitolist[i])] = trimesh_vtk.mesh_actor(var_iterator_unkdend['mito_mesh_'+str(mito_query_unkdend_mitolist[i])], opacity=mito_opac_unkdend, color=(mito_color_unkdend))
locals().update(actor_iterator_unkdend)

# update dictionary for the render actors code below
actor_iterator = {}
actor_iterator['mesh_actor'] = mesh_actor
actor_iterator.update(actor_iterator_basal)
actor_iterator.update(actor_iterator_apical)
actor_iterator.update(actor_iterator_axonal)
actor_iterator.update(actor_iterator_somatic)
actor_iterator.update(actor_iterator_unknown)
actor_iterator.update(actor_iterator_unkdend)
locals().update(actor_iterator)

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=200) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000021710864B90) at 0x000002177233EE88>

In [31]:
#actor_iterator

In [32]:
#type(actor_iterator.values())

In [35]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mesh.centroid, backoff=190)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000002179C7E1C30) at 0x000002177233EAC8>

In [36]:
cellid

648518346349539853

In [37]:
#pyr_list

# Neuroglancer visualizer

In [38]:
lookup = cellid

In [39]:
query = mito[mito["cellid"] == lookup]

In [40]:
len(query)

945

In [41]:
query_list = query["mito_id"].to_list()
spacer = "%22%2C%22"
query_list_join = spacer.join(map(str,query_list))
baseURL = 'https://neuromancer-seung-import.appspot.com/'
nglink_a = '#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22'
nglink_b = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220%22%2C%22type%22:%22segmentation%22%2C%22segments%22:%5B%22'
nglink_c = '%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22mitochondria%22%7D%2C%7B%22source%22:%22precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-nuclei/seg%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22nuclei%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B83222.921875%2C52981.34765625%2C824.9962768554688%5D%7D%7D%2C%22zoomFactor%22:383.0066650796121%7D%2C%22perspectiveOrientation%22:%5B-0.00825042650103569%2C0.06130112707614899%2C-0.0012821174459531903%2C0.9980843663215637%5D%2C%22perspectiveZoom%22:3618.7659948513424%2C%22showSlices%22:false%2C%22selectedLayer%22:%7B%22layer%22:%22mitochondria%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%223d%22%2C%22orthographicProjection%22:true%7D%7D'
neuroglancerURL = baseURL + nglink_a + str(lookup) + nglink_b + query_list_join + nglink_c
print(neuroglancerURL)

https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22EM%22%7D%2C%7B%22source%22:%22precomputed://gs://microns_public_datasets/pinky100_v185/seg%22%2C%22type%22:%22segmentation%22%2C%22selectedAlpha%22:0.51%2C%22objectAlpha%22:0.15%2C%22segments%22:%5B%22648518346349539853%22%5D%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22cell_segmentation_v185%22%7D%2C%7B%22source%22:%22precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky100-clefts/mip1_d2_1175k%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22synapses%22%7D%2C%7B%22source%22:%22precomputed://https://tigerdata.princeton.edu/sseung-archive/pinky1

In [42]:
query

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
259,3098978,30152,95960,67402,621,95872,67374,614,96072,67434,631,648518346349539853,383840,269608,24840,0.486600,3.053596,Axonal
525,3170590,125152,97918,50832,954,97666,50600,937,98176,51040,968,648518346349539853,391672,203328,38160,1.878677,10.200201,Somatic
717,3970091,332188,114540,58920,1279,114014,58698,1272,114964,59126,1287,648518346349539853,458160,235680,51160,4.249911,16.760998,Basal
969,3620264,1832860,109466,53436,650,107652,52536,535,111472,54400,714,648518346349539853,437864,213744,26000,17.480647,38.312640,Basal
1003,3515725,1089960,106290,59742,1079,105440,58126,968,106992,61020,1173,648518346349539853,425160,238968,43160,12.635230,40.912546,Basal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210175,3175427,697804,98004,52030,677,97756,51616,641,98292,52408,729,648518346349539853,392016,208120,27080,8.710557,32.703864,Somatic
210423,2264009,768908,82692,50680,689,81984,50518,573,83166,50866,786,648518346349539853,330768,202720,27560,8.105520,21.703113,Basal
210470,3068752,785008,96824,52592,156,96696,52254,114,96970,53020,221,648518346349539853,387296,210368,6240,7.085349,13.907971,Somatic
210578,3511865,836376,104410,56030,2010,103808,55838,1891,105332,56156,2125,648518346349539853,417640,224120,80400,8.365461,20.164838,Basal


In [43]:
query.sort_values(by='mito_vx', ascending=False)

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm,surface_area,complexityindex,compartment
2730,846898,5241524,69066,37174,1711,57952,35660,1503,79204,40520,1814,648518346349539853,276264,148696,68440,63.240397,221.818001,Basal
22179,3599973,3377304,113472,41788,1675,107570,39978,1384,119166,44296,2148,648518346349539853,453888,167152,67000,41.869051,155.048379,Basal
79301,3214422,3205284,100036,65934,105,98990,61590,21,100752,70056,193,648518346349539853,400144,263736,4200,33.378725,87.217650,Basal
107770,3109814,2925992,97696,71850,694,97028,68568,608,98728,75582,810,648518346349539853,390784,287400,27760,27.976834,61.627937,Basal
20157,2371418,2922848,86616,43976,1499,85220,41896,1168,87786,46970,1845,648518346349539853,346464,175904,59960,30.313414,78.563398,Basal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12226,2746120,1464,91326,56156,1615,91314,56116,1614,91342,56188,1618,648518346349539853,365304,224624,64600,0.072949,4.364229,Basal
11611,3064205,1376,96664,50366,681,96650,50352,680,96680,50386,683,648518346349539853,386656,201464,27240,0.044357,1.110671,Somatic
11561,3068729,1368,96690,52016,994,96672,52002,992,96718,52032,997,648518346349539853,386760,208064,39760,0.057651,2.467077,Somatic
6215,3069566,1356,97638,52794,744,97620,52786,743,97658,52808,747,648518346349539853,390552,211176,29760,0.046990,1.359613,Somatic


# Center vtk view at centroid of mitoid of interest

In [44]:
mitoid_query = 846898

In [45]:
pull_centroid = mito_querydf.reset_index()
pull_centroid = pull_centroid.index[pull_centroid['mito_id'] == mitoid_query].to_list()
pull_centroid[0]

17

In [46]:
mito_mesh_centroid_dict = {}
mito_seg_id = mitoid_query
mito_mesh_file = os.path.join(mito_mesh_dir, str(mito_seg_id)+'.h5')
mito_mesh_centroid_dict['mito_mesh_centroid_' + str(pull_centroid[0])] = mito_mm.mesh(filename = mito_mesh_file)
locals().update(mito_mesh_centroid_dict)

In [47]:
mito_mesh_centroid_dict

{'mito_mesh_centroid_17': <trimesh.Mesh(vertices.shape=(18175, 3), faces.shape=(36350, 3))>}

In [48]:
mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid

array([276898.76239413, 148945.04219488,  68380.47984756])

In [49]:
backoff_value = 20

#creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid, backoff=backoff_value) # 150

#render the actors, will open a pop up python window
trimesh_vtk.render_actors(actor_iterator.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x000002179C7DDD10) at 0x000002177233EF48>

In [ ]:
# save image to disk
# uncomment to save

#camera = trimesh_vtk.oriented_camera(mito_mesh_centroid_dict.get('mito_mesh_centroid_'+str(pull_centroid[0])).centroid, backoff=backoff_value)

#trimesh_vtk.render_actors(actor_iterator.values(),
#                          filename='vtk_images/'+str(cellid)+'_'+str(mitoid_query)+'.png',
#                          do_save=True,
#                          scale=6,
#                          camera=camera
#                         )